In [2]:
!pip install dlib

  Using cached dlib-19.24.2.tar.gz (11.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.2-cp311-cp311-win_amd64.whl size=2838471 sha256=10c2fd48e42a4e002557220135806bd4799c16b155ef1da337e95967284accc4
  Stored in directory: c:\users\furqan970\appdata\local\pip\cache\wheels\61\05\62\44b0bf18a0f8f9a0d65337b11237ecf12926d0d6e3807500bb
Successfully built dlib


In [2]:
import cv2
import numpy as np
import dlib
from math import hypot
import pyautogui

In [4]:
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
def midpoint(p1, p2):
    return int((p1.x+p2.x)/2),int((p1.y+p2.y)/2) #pixel value cannot be in decimals
font = cv2.FONT_HERSHEY_PLAIN
def get_blinking_ratio(eye_points,facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    centre_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    centre_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))
#     hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
#     ver_line = cv2.line(frame, centre_top, centre_bottom, (0,255,0), 2)
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_length = hypot((centre_top[0] - centre_bottom[0]), (centre_top[1] - centre_bottom[1]))
    ratio = hor_line_length/ver_line_length
    return ratio 
def get_gaze_ratio(eye_points, facial_landmarks):
    left_eye_region = np.array([(landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                   (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                   (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                   (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                   (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                   (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)
    #print(left_eye_region)
    #cv2.polylines(frame, [left_eye_region], True,(0,0,255), 2)
    height, width, _ = frame.shape
    mask = np.zeros((height,width), np.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)
    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])
    gray_eye = eye[min_y: max_y, min_x: max_x]
    #gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    #print(type(threshold_eye)) 
    height,width = threshold_eye.shape
    left_side_threshold = threshold_eye[0: height, 0: int(width/2)]
    left_side_white = cv2.countNonZero(left_side_threshold)
    right_side_threshold = threshold_eye[0: height, int(width/2): width]
    right_side_white = cv2.countNonZero(right_side_threshold)
    # Handle zero division
    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:   
        gaze_ratio = left_side_white/right_side_white
    return gaze_ratio
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        #x, y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x,y), (x1,y1), (0,255,0), 2)
        landmarks = predictor(gray, face)
#         x = landmarks.part(36).x
#         y = landmarks.part(36).y
#         cv2.circle(frame, (x,y), 3, (0,0,255), 2)
        #Blink Detection 
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41], landmarks)
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio)/2
        if blinking_ratio > 4:
            cv2.putText(frame, "BLINKING", (50, 150), font, 3, (255,0,0))
        elif blinking_ratio > 6:
            cv2.putText(frame, "CLICK", (50, 150), font, 3, (255,0,0))
            pyautogui.click()
        #Gaze Detection
        gaze_ratio_left_eye = get_gaze_ratio([36,37,38,39,40,41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42,43,44,45,46,47], landmarks)
        gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2
        new_frame = np.zeros((500, 500, 3), np.uint8)
        if gaze_ratio < 1:
            cv2.putText(frame, "RIGHT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (0, 0,255)
            pyautogui.moveRel(10, 0)
        elif 1 < gaze_ratio < 2:
            cv2.putText(frame, "CENTER", (50, 100), font, 3, (0,0,255), 3)
            new_frame[:] = (0, 255, 0)
            pyautogui.moveRel(0, 0)
        else :
            cv2.putText(frame, "LEFT", (50, 100), font, 3, (0,0,255), 3)
            new_frame[:] = (255, 0, 0)
            pyautogui.moveRel(-10, 0)
        #cv2.putText(frame, str(left_side_white), (50,100), font, 2, (0, 0, 255), 3)
        #cv2.putText(frame, str(right_side_white), (50,150), font, 2, (0, 0, 255), 3)
        #cv2.putText(frame, str(gaze_ratio), (50,100), font, 2, (0, 0, 255), 3)
#         threshold_eye=cv2.resize(threshold_eye, None, fx=5, fy = 5)
#         eye=cv2.resize(gray_eye, None, fx=5, fy = 5)
#         #cv2.imshow("Eye", eye)
#         cv2.imshow("Threshold", threshold_eye)
#         cv2.imshow("Left", left_side_threshold)
#         cv2.imshow("right", right_side_threshold)
        #cv2.imshow("Left Eye", left_eye)
    cv2.imshow("Frame", frame)
    cv2.imshow("New Frame", new_frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()